In [ ]:
import pandas as pd

df = pd.read_excel("files\GVC - Padrões Globais.xlsx")
df

In [ ]:
codeTermDict: dict[str, str] = {}

for idx, value in df["Código"].items():
    parts = value.split()
    
    while len(parts) < 7:
        parts.append("00")
    
    newValue = "-".join(parts)  # '3E-02-00-00-00-00-00'

    term = df.at[idx, "Termo"]
    codeTermDict[newValue] = term

import json
import os

os.makedirs("files", exist_ok=True)
file_path = os.path.join("files", "nested_codes.json")

with open(file_path, "w", encoding="utf-8") as f:
    json.dump(codeTermDict, f, indent=2, ensure_ascii=False)

codeTermDict

In [ ]:
import re
from typing import List
from Types import GvcObject, Property


IGNORED_KEYWORDS = [
    "id", "guid", "handle", "path", "element", "category id", "family id",
    "internal", "level", "host", "parameter id", "unique id"
]

NUMERIC_PATTERN = re.compile(r"^\d+([.,]\d+)?$")


def is_useful_property(prop: Property) -> bool:
    if not prop or not prop.Info:
        return False

    if len(prop.Info.strip()) < 3:
        return False

    if NUMERIC_PATTERN.match(prop.Info.strip()):
        return False

    name = (prop.Name or "").lower()
    cat = (prop.Category or "").lower()
    if any(kw in name for kw in IGNORED_KEYWORDS):
        return False
    if any(kw in cat for kw in IGNORED_KEYWORDS):
        return False

    return True


def extract_useful_properties(items: List[GvcObject]) -> List[dict]:
    results = []

    for obj in items:
        useful_props = [p for p in obj.Properties if is_useful_property(p)]

        if useful_props:
            results.append({
                "object_id": obj.ObjectId,
                "name": obj.Name,
                "type": obj.Type,
                "source": obj.SourceFile,
                "useful_properties": [p.__todict__() for p in useful_props]
            })

    return results


In [ ]:
from Db import *

items = DBManager._getItems(limit=10000)